In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import pickle

In [2]:
d1 = "1981-01-01"
d2 = "2020-12-31"

# Read Factors

- make ``F`` as a dictionary

In [3]:
f = pd.read_csv("../data/factor/ff_factors.csv", index_col=0)
f.index = pd.date_range(start="1963-07-31",end="2021-02-28",freq='M')
f = f.loc[d1:d2]
f = f/100
f

,Mkt-RF,SMB,HML,RMW,CMA,RF,MOM
1981-01-31,-0.0504,0.0338,0.0683,-0.0354,0.0435,0.0104,-0.0794
1981-02-28,0.0057,-0.0049,0.0100,0.0019,0.0225,0.0107,-0.0139
1981-03-31,0.0356,0.0307,0.0069,-0.0234,-0.0053,0.0121,0.0073
1981-04-30,-0.0211,0.0460,0.0229,0.0082,0.0120,0.0108,-0.0095
1981-05-31,0.0011,0.0245,-0.0047,0.0033,-0.0160,0.0115,0.0375
...,...,...,...,...,...,...,...
2020-08-31,0.0763,-0.0094,-0.0294,0.0427,-0.0144,0.0001,0.0051
2020-09-30,-0.0363,0.0007,-0.0251,-0.0115,-0.0177,0.0001,0.0305
2020-10-31,-0.0210,0.0476,0.0403,-0.0060,-0.0053,0.0001,-0.0303
2020-11-30,0.1247,0.0675,0.0211,-0.0278,0.0105,0.0001,-0.1225


In [4]:
F = {
    "CAPM": f[["Mkt-RF"]],
    "FF3":  f[["Mkt-RF","SMB","HML"]],
    "FF5":  f[["Mkt-RF","SMB","HML","RMW","CMA"]],
    "FF6":  f[["Mkt-RF","SMB","HML","RMW","CMA","MOM"]]
}
RF = f[["RF"]]

# Read Portfolio Returns

- minus risk free rate

In [5]:
ff25 = pd.read_csv("../data/portfolio/FF25.csv", index_col=0)
ff25.index = pd.date_range(start="1926-07-31",end="2021-03-31",freq='M')
ff25 = ff25.loc[d1:d2]
ff25 = ff25/100
ff25['RF'] = RF["RF"]
for i in ff25.columns[:-1]:
    ff25[i] = ff25[i] - ff25["RF"]
del ff25["RF"]
ff25


,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
1981-01-31,-0.055713,-0.012004,0.009615,0.004854,0.021836,-0.059579,-0.010723,-0.011928,0.007083,0.023116,...,-0.077319,-0.046673,-0.013693,-0.005039,0.003943,-0.071126,-0.085457,-0.074981,-0.010037,0.009464
1981-02-28,-0.044100,-0.009855,0.000755,0.010513,0.013616,-0.015052,0.006063,0.017782,0.023812,0.027143,...,0.003582,0.017735,0.034421,0.002093,0.027327,0.022088,0.008993,-0.016489,0.020446,-0.016853
1981-03-31,0.060772,0.083000,0.074279,0.079229,0.061365,0.067804,0.069669,0.068041,0.066793,0.063962,...,0.076795,0.052515,0.075006,0.061562,0.064823,0.017621,0.019556,-0.010361,0.022230,0.069573
1981-04-30,0.019735,0.036720,0.025753,0.036525,0.044092,0.005351,0.025912,0.025813,0.026331,0.027051,...,-0.010527,-0.020591,-0.005563,-0.007822,0.006336,-0.037968,-0.053810,-0.037873,0.010439,-0.040374
1981-05-31,0.038741,0.038799,0.032262,0.013419,0.016525,0.050967,0.011507,0.015348,-0.008609,0.011735,...,0.024675,0.007855,0.009175,-0.000762,0.022051,0.004842,-0.033121,-0.031763,0.026669,-0.005979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.035670,0.055853,0.040523,0.055002,0.052459,0.076433,0.070196,0.046803,0.032090,0.091388,...,0.043670,0.036393,0.046300,0.046432,0.036272,0.107804,0.064340,0.024204,0.027602,0.034731
2020-09-30,-0.012799,-0.003731,-0.045472,-0.036109,-0.029951,0.003582,-0.049998,-0.051121,-0.069621,-0.051715,...,0.010041,-0.026547,-0.035114,-0.030525,-0.048866,-0.045608,-0.035745,-0.012623,-0.046809,-0.061218
2020-10-31,-0.026940,-0.012818,-0.001701,0.030712,0.007194,-0.022918,0.040625,0.023629,0.056444,0.059473,...,0.013598,0.020623,0.018701,0.018790,0.019996,-0.045811,-0.000070,-0.000969,-0.023344,0.009783
2020-11-30,0.254503,0.216752,0.198199,0.205256,0.239505,0.213781,0.190604,0.179496,0.190474,0.193057,...,0.138104,0.128181,0.159509,0.157006,0.203974,0.107157,0.098447,0.144505,0.161554,0.219680


In [6]:
me10 = pd.read_csv("../data/portfolio/size_sort_port.csv", index_col=0)
me10 = me10[['Lo 10', 'Dec-02', 'Dec-03', 'Dec-04', 'Dec-05', 
                'Dec-06','Dec-07', 'Dec-08', 'Dec-09', 'Hi 10']]
me10.index = pd.date_range(start="1926-07-31",end="2021-04-30",freq='M')
me10 = me10.loc[d1:d2]
me10 = me10/100
me10['RF'] = RF["RF"]
for i in me10.columns[:-1]:
    me10[i] = me10[i] - me10["RF"]
del me10["RF"]
me10


,Lo 10,Dec-02,Dec-03,Dec-04,Dec-05,Dec-06,Dec-07,Dec-08,Dec-09,Hi 10
1981-01-31,-0.0107,-0.0216,-0.0195,-0.0254,-0.0207,-0.0310,-0.0391,-0.0390,-0.0491,-0.0656
1981-02-28,-0.0151,-0.0140,-0.0028,0.0129,0.0071,0.0113,0.0047,0.0176,0.0021,0.0082
1981-03-31,0.0596,0.0683,0.0662,0.0655,0.0586,0.0724,0.0736,0.0670,0.0615,0.0040
1981-04-30,0.0249,0.0317,0.0144,0.0267,0.0129,0.0099,-0.0065,-0.0139,-0.0173,-0.0383
1981-05-31,0.0228,0.0238,0.0235,0.0222,0.0163,0.0183,0.0174,0.0063,0.0056,-0.0114
...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.0215,0.0733,0.0519,0.0708,0.0572,0.0515,0.0480,0.0413,0.0301,0.0906
2020-09-30,-0.0079,-0.0362,-0.0512,-0.0361,-0.0289,-0.0370,-0.0286,-0.0075,-0.0164,-0.0429
2020-10-31,-0.0194,0.0124,0.0369,0.0210,0.0346,0.0297,0.0247,0.0020,0.0083,-0.0348
2020-11-30,0.2343,0.2175,0.2104,0.1887,0.1898,0.1644,0.1566,0.1375,0.1328,0.1126


In [7]:
P = {
    "ME10": me10,
    "FF25": ff25
}

# Save Data as ``.pickle``

In [8]:
pickle.dump( [P,F], open( "../data/save.p", "wb" ) )